<img style="float: left;" src="https://cdn.pixabay.com/photo/2016/12/07/09/45/dna-1889085__340.jpg" width=10%> <h1> Application of AI to Discover Novel Binding of Small Molecules </h1>

---------
### Sample Dataset for Testing Purposes

##### Here we create a sample dataset for two reasons:
- to get a better understanding of the structure of the data
- test any sample code for validity

##### Structure of sample dataset:
1. A dataframe consisting of 50 genes and 1020 profiles [50 x 1020]
2. Columns are a combination of drug, replicate, time, concentration, probe_location, cell type. For the purposes of this project only drug and replicate matters in terms of training. So the column name will be structured as
"*drug + replicate id + unique characters that represent time, concentration, probe_location and cell type*"
3. 20 columns consist of control genes or 'control probes'. Columns are labelled control_x where x is a number from 1 to 20
3. Dataset consists of 25 drugs with 4 replicates and 10 combinations of time, concentration, probe_location and cell type

| Feature      | Quantity | Represented By |
| ----------- | ----------- | ----------- |
| Drug      | 25       | Alphabets A-Y |
| Replicate   | 4        | Numbers 1-4 |
| Other features   | 10        | Random String of length 3 |

***R_3_xcv*** represents a profile of drug 'R', of replicate 3, with other features coresponding to 'xcv'

##### Construction of Sample Dataset

In [17]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from itertools import chain

import keras
from keras.datasets import reuters
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers.noise import AlphaDropout
from keras.preprocessing.text import Tokenizer
from keras.layers import Layer
from tensorflow.python.keras import backend as K

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
genes = ['gene'+str(a) for a in range(50)]
drugs = [chr(a) for a in range(65, 90)]
replicates = [str(a) for a in range(1, 5)]
other_features = set()

while len(other_features)!=10:
    rand_string = "". join([str(chr(int(random.random()*100)%26+97)) for a in range(3)])
    other_features.add(rand_string)

In [3]:
columns = ["_".join([a,b,c]) for a in drugs for b in replicates for c in other_features]
# columns = ["control_"+str(a+1) for a in range(20)] + columns

In [4]:
data = pd.DataFrame(2*np.random.rand(50, len(columns))-1, index=genes, columns=columns)
data.columns = columns
data.fillna(random.random(), inplace = True)
data.shape

(50, 1000)

In [5]:
data.head()

,A_1_kky,A_1_yjn,A_1_ftu,A_1_qvc,A_1_lsf,A_1_ozh,A_1_ebc,A_1_toa,A_1_klb,A_1_ima,...,Y_4_kky,Y_4_yjn,Y_4_ftu,Y_4_qvc,Y_4_lsf,Y_4_ozh,Y_4_ebc,Y_4_toa,Y_4_klb,Y_4_ima
gene0,0.565431,0.454328,-0.997431,-0.147522,-0.266993,0.647802,0.327706,-0.782721,-0.251218,-0.253343,...,0.677855,-0.538512,-0.657175,-0.185452,0.964592,0.125108,-0.621859,-0.032930,0.802544,0.436068
gene1,-0.339680,-0.703011,0.547694,-0.402608,0.303426,-0.064423,0.243335,0.967988,-0.375953,-0.889101,...,0.992498,0.673060,0.677454,-0.796117,-0.391574,-0.497911,0.977625,0.926948,-0.047017,-0.868945
gene2,0.727901,-0.196407,-0.183411,-0.315536,0.898675,-0.309319,0.357032,-0.706039,0.548331,0.659212,...,-0.559336,0.180941,0.008483,0.430328,-0.716242,-0.871301,0.443298,0.431096,0.908664,0.866675
gene3,0.156712,0.414263,-0.288991,-0.511516,0.776627,-0.032848,-0.478349,-0.809679,0.891934,0.771603,...,0.006165,-0.286369,-0.761032,-0.409968,0.562998,0.600992,0.058420,-0.533900,0.634965,0.371435
gene4,0.567214,-0.212149,-0.375861,0.516339,-0.076705,0.385077,-0.640458,0.234052,-0.758381,0.065517,...,-0.230065,0.690195,-0.107643,-0.132128,-0.290238,-0.240014,0.150119,-0.042335,0.998942,-0.107498


##### Classifying Columns
A label needs to be assigned to each class. This can be done at the biological replicate level or the perturbagen level. We create classifications for each of these.

In [6]:
perturbagen_class = [int(a/25) for a in range(1000)]
replicate_class = [10*a+c for a in range(25) for b in range(4) for c in range(10)]

##### Creating the dataset

In [7]:
#transpose data
workingdata = data.transpose()
workingdata.head()

,gene0,gene1,gene2,gene3,gene4,gene5,gene6,gene7,gene8,gene9,...,gene40,gene41,gene42,gene43,gene44,gene45,gene46,gene47,gene48,gene49
A_1_kky,0.565431,-0.339680,0.727901,0.156712,0.567214,0.854648,-0.912212,0.109724,0.651533,0.056977,...,0.978305,-0.888512,-0.391051,-0.993740,-0.219221,0.131645,0.409954,-0.801225,-0.842967,0.366684
A_1_yjn,0.454328,-0.703011,-0.196407,0.414263,-0.212149,-0.475095,0.768395,0.798914,0.464159,0.863646,...,-0.673128,0.216370,0.264719,0.843114,0.429802,-0.872912,-0.339984,-0.171954,0.854678,-0.788994
A_1_ftu,-0.997431,0.547694,-0.183411,-0.288991,-0.375861,-0.356187,0.433200,-0.285618,-0.122269,-0.676739,...,0.465353,0.481015,0.268942,-0.838261,0.671298,0.771700,0.190407,0.265288,-0.391407,-0.659839
A_1_qvc,-0.147522,-0.402608,-0.315536,-0.511516,0.516339,0.566360,0.071008,0.604673,0.033551,0.368689,...,0.417015,0.603839,0.000614,0.530540,0.296515,-0.220551,0.491672,-0.762185,0.873178,-0.402150
A_1_lsf,-0.266993,0.303426,0.898675,0.776627,-0.076705,0.251014,0.350783,0.148486,-0.297146,0.228995,...,0.416483,0.689567,0.155130,0.414486,0.597038,-0.641432,0.481483,0.751060,-0.971406,0.064329


In [8]:
X=workingdata

In [92]:
def get_batch(batch_size):
    rng = np.random
    #perturbagens, profiles in each perturbagen, dimension of each profile
    n_classes, n_examples, dimx, dimy = 25, 40, 50, 1 
    
    # randomly sample several classes to use in the batch
    perturbagens = rng.choice(n_classes,size=(batch_size,),replace=False)
    
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, dimx)) for i in range(2)]
    
    # initialize vector for the targets
    targets=np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    #print(perturbagens)
    
    for i in range(batch_size):
        pert = perturbagens[i]
        idx_1 = rng.randint(0, n_examples)
        #print(pert, idx_1)
        #print(X.iloc[n_examples*(pert-1)+idx_1,:].shape)
        
        pairs[0][i,:] = X.iloc[n_examples*(pert-1)+idx_1,:]#.values.reshape(dimx,dimy)
        #print(pairs[0].shape)
        idx_2 = rng.randint(0, n_examples)
        
        # pick images of same class for 1st half, different for 2nd
        pert2=pert
        if i < batch_size // 2:
            pert2 = rng.choice(list(range(pert)) + list(range(pert+1, n_classes)))

        pairs[1][i,:] = X.iloc[n_examples*(pert2-1)+idx_1,:]#.values.reshape(dimx,dimy)

    return np.asarray(pairs), np.asarray(targets)

In [93]:
p, t = get_batch(10)
p.shape

(2, 10, 50)

In [77]:
def generate(batch_size, s="train"):
    """
    a generator for batches, so model.fit_generator can be used.
    """
    while True:
        pairs, targets = get_batch(batch_size,s)
        yield (pairs, targets)

In [78]:
def make_oneshot_task( N):
    rng = np.random
    n_classes, n_examples, dimx, dimy = 25, 40, 50, 1 
    
    indices = rng.randint(0, n_examples,size=(N,))
    perturbagens = rng.choice(range(n_classes),size=(N,),replace=False)   
    true_category = perturbagens[0]
    
    ex1, ex2 = rng.choice(n_examples,replace=False,size=(2,))
    
    test_image = np.asarray([X.iloc[(true_category*n_examples)+ex1]]*N)
    
    support_set = np.asarray(X.iloc[perturbagens*n_examples+indices,:])
    support_set[0,:] = X.iloc[(true_category*n_examples+ex2)]
    targets = np.zeros((N,))
    targets[0] = 1
    pairs = [test_image,support_set]
    return pairs, targets

In [79]:
p,t = make_oneshot_task(4)

In [80]:
def test_oneshot(model, N, k,verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

In [81]:
def nearest_neighbour_correct(pairs,targets):
    """returns 1 if nearest neighbour gets the correct answer for a one-shot task
        given by (pairs, targets)"""
    L2_distances = np.zeros_like(targets)
    for i in range(len(targets)):
        L2_distances[i] = np.sum(np.sqrt(pairs[0][i]**2 - pairs[1][i]**2))
    if np.argmin(L2_distances) == np.argmax(targets):
        return 1
    return 0

  
def test_nn_accuracy(N_ways,n_trials):
    """Returns accuracy of NN approach """
    print("Evaluating nearest neighbour on {} unique {} way one-shot learning tasks ...".format(n_trials,N_ways))
    n_right = 0
    
    for i in range(n_trials):
        pairs,targets = make_oneshot_task(N_ways,"val")
        correct = nearest_neighbour_correct(pairs,targets)
        n_right += correct
    return 100.0 * n_right / n_trials

In [82]:
max_words = 50
batch_size = 16
epochs = 40

def create_network(n_dense=6,
                   dense_units=16,
                   activation='selu',
                   dropout=AlphaDropout,
                   dropout_rate=0.1,
                   kernel_initializer='lecun_normal',
                   optimizer='adam',
                   num_classes=1,
                   max_words=max_words):
    
    model = Sequential()
    model.add(Dense(dense_units, input_shape=(max_words,),
                    kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(dropout(dropout_rate))

    for i in range(n_dense - 1):
        model.add(Dense(dense_units, kernel_initializer=kernel_initializer))
        model.add(Activation(activation))
        model.add(dropout(dropout_rate))

    #model.add(Dense(num_classes))
    #model.add(Activation('softmax'))
    return model

In [83]:
network = {
    'n_dense': 10,
    'dense_units': 16,
    'activation': 'selu',
    'dropout': AlphaDropout,
    'dropout_rate': 0.1,
    'kernel_initializer': 'lecun_normal',
    'optimizer': 'sgd',
    'num_classes':40
}

In [84]:
shared_model = create_network(**network)

In [85]:
class ManDist(Layer):
    
    # initialize the layer, No need to include inputs parameter!
    def __init__(self, **kwargs):
        self.result = None
        super(ManDist, self).__init__(**kwargs)

    # input_shape will automatic collect input shapes to build layer
    def build(self, input_shape):
        super(ManDist, self).build(input_shape)

    # This is where the layer's logic lives.
    def call(self, x, **kwargs):
        self.result = K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True)
        return self.result

    # return output shape
    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)

In [86]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) +
                  (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

In [87]:
left_input = Input(shape=(max_words,))
right_input = Input(shape=(max_words,))

In [88]:
malstm_distance = ManDist()([shared_model(left_input), shared_model(right_input)])
model = Model(inputs=[left_input, right_input], outputs=[malstm_distance])
model.compile(loss=contrastive_loss, optimizer="adam", metrics=['accuracy'])

In [97]:
import os
model_path = './weights/'

In [99]:
import time 
# Hyper parameters
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 10
n_iter = 2000 # No. of training iterations
N_way = 20 # how many classes for testing one-shot tasks
n_val = 250 # how many one-shot tasks to validate on
best = -1

print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    (inputs,targets) = get_batch(batch_size)
    loss = model.train_on_batch([inputs[0],inputs[1]], targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss)) 
        val_acc = test_oneshot(model, N_way, n_val, verbose=True)
        #model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc
    

Starting training process!
-------------------------------------

 ------------- 

Time for 200 iterations: 0.028206419944763184 mins
Train Loss: [1.3858105, 0.2]
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 5.2% 20 way one-shot learning accuracy 

Current best: 5.2, previous best: -1

 ------------- 

Time for 400 iterations: 0.06502366860707601 mins
Train Loss: [0.8624792, 0.3]
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 6.4% 20 way one-shot learning accuracy 

Current best: 6.4, previous best: 5.2

 ------------- 

Time for 600 iterations: 0.10598047971725463 mins
Train Loss: [1.4148902, 0.1]
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 5.6% 20 way one-shot learning accuracy 


 ------------- 

Time for 800 iterations: 0.14987645149230958 mins
Train Loss: [1.1376113, 0.4]
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 4.4%